In [ ]:
# Import necessary libraries (taken from the multi_analysis script)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import mne
from scipy import signal, stats
import pandas as pd
import logging
from typing import Dict, List, Tuple, Optional
from matplotlib.gridspec import GridSpec
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import json
import warnings

# Import custom modules
from imagined_vs_actual_analysis_new import ImaginedVsActualAnalyzer, COLORS

# **Code for calculating Power Spectral Entropy Differences between Imagined and Actual Movements**

In [13]:
def spectral_entropy(psd, eps=1e-12):
    psd = psd + eps
    psd /= psd.sum()
    H = -np.sum(psd * np.log2(psd))
    return H / np.log2(len(psd))

def entropy_from_epochs(
    epochs: mne.Epochs,
    fmin: float = 8.0,
    fmax: float = 30.0
) -> np.ndarray:
    # Compute mean spectral entropy per epoch across channels.
    psd = epochs.compute_psd(
        method="welch",
        fmin=fmin,
        fmax=fmax,
        average="mean",
        verbose=False
    )
    # Shape: (n_epochs, n_channels, n_freqs)
    psds = psd.get_data()
    entropies = np.zeros((psds.shape[0], psds.shape[1]))
    for e in range(psds.shape[0]):
        for ch in range(psds.shape[1]):
            entropies[e, ch] = spectral_entropy(psds[e, ch])

    # Return mean entropy per epoch (averaged across channels)
    return entropies.mean(axis=1)

def entropy_modulation(task_epochs, baseline_raw):
    baseline_epochs = mne.make_fixed_length_epochs(
        baseline_raw,
        duration=2.0,
        overlap=1.0,
        preload=True,
        verbose=False
    )
    H_task = entropy_from_epochs(task_epochs).mean()
    H_base = entropy_from_epochs(baseline_epochs).mean()
    return H_base - H_task  # Positive = more structured during task



# Test run on 1 person 
analyzer = ImaginedVsActualAnalyzer(subject_id="S003")
analyzer.load_and_preprocess_data()
baseline_raw = analyzer.data['baseline_eyes_open']['raw']
task_key = "Left/Right Fist 1_imagined"
task_epochs = analyzer.data[task_key]['epochs'][:10]
entropy_score = entropy_modulation(task_epochs, baseline_raw)
print(f"Motor imagery entropy modulation: {entropy_score:.3f}")


2026-01-08 18:32:11,700 - INFO - ======================================================================
2026-01-08 18:32:11,700 - INFO - LOADING DATA FOR S003 - IMAGINED VS ACTUAL ANALYSIS
2026-01-08 18:32:11,701 - INFO - ======================================================================
2026-01-08 18:32:11,701 - INFO - 
Loading baseline: eyes_open (Run 1)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-08 18:32:11,719 - INFO -   ✓ Loaded baseline: 61.0s, 3 channels
2026-01-08 18:32:11,719 - INFO - 
Loading baseline: eyes_closed (Run 2)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-08 18:32:11,789 - INFO -   ✓ Loaded baseline: 61.0s, 3 channels
2026-01-08 18:32:11,790 - INFO - 
Loading Left/Right Fist 1_real (Run 3)


...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-08 18:32:12,108 - INFO -   ✓ Loaded: 29 epochs, 3 channels
2026-01-08 18:32:12,108 - INFO - 
Loading Fists/Feet 3_imagined (Run 14)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2026-01-08 18:32:12,137 - INFO -   ✓ Loaded: 29 epochs, 3 channels
2026-01-08 18:32:12,137 - INFO - 
✓ Successfully loaded 14 datasets


Motor imagery entropy modulation: -0.053
